# Matrix Factorization

We will implement matrix factorization here, both manually and maybe through TensorFlow.

## Theory Behind Matrix Factorization: Problem Statement

We will give a brief rundown, nothing too complicated.

The main **mathematical** statement that we will tackle with matrix factorization is as follows:

We have some matrix $A$ (this could be a tabular dataset or anything else). Now the *caveat* is that $A$ has **missing entries** that do not know about. We want to find two matrices $U$ and $V$, so that their product:

$$U \cdot V^{T} \approx A$$

So really, in terms of an optimization problem, we want to minimize each entry of the matrix $E$:

$$E = A - U\cdot V^{T}$$

It really helps to write this in index notation to represent each component so that we are not dealing with matrices directly. Let $E_{ij} := e_{ij}$, then, writing out the above expression in matrix form, we see that:

$$e_{ij} = a_{ij} - \sum_{k}u_{ik}v_{jk}^{T} = a_{ij} - \sum_{k}u_{ik}v_{kj}$$

We are now only working with real numbers! We can now adopt the traditional MSE loss for each entry.

$$e_{ij}^{2} = \left(a_{ij} - \sum_{k}u_{ik}v_{kj}\right)^{2}$$

or to make it look simpler, we can write $\hat{a}_{ij} = \sum_{k}u_{ik}v_{kj}$, and we now have:

$$e_{ij}^{2} = (a_{ij} - \hat{a}_{ij})^{2}$$

We are now minimizing this expression for all $(i,j) \in rows(A), cols(A))$. **For all intents and purposes**, this is all we really need to know to continue.

## Theory of Matrix Factorization: Machine Learning

However, we can continue our derivation of our equations, as this will help us in our implementation. We take the gradient (derivative) of our loss above, with respect to the nonzero components of both $U$ and $V$ (**Notice that we are not training weights and biases, but just the entries of the matrices $U$ and $V$!!!**):

$$\frac{\partial e_{ij}^{2}}{\partial u_{ik}} = \frac{\partial e_{ij}^{2}}{\partial e_{ij}}\frac{\partial e_{ij}}{\partial u_{ik}} = -2e_{ij}v_{kj} $$

Likewise, for the other component:

$$\frac{\partial e_{ij}^{2}}{\partial v_{ik}} = -2e_{ij}u_{ik}$$

We now do the **optimization step** (i.e. the *Gradient Descent Step*):

$$u_{ik} \longmapsto u_{ik} - \lambda \frac{\partial e_{ij}^{2}}{\partial u_{ik}} = u_{ik} + 2\lambda e_{ij}v_{kj}$$

$$v_{kj} \longmapsto v_{kj} - \lambda \frac{\partial e_{ij}^{2}}{\partial v_{kj}} = v_{kj} + 2\lambda e_{ij}u_{ik}$$

## Business Use of Matrix Factorization

The business problem we are trying to solve is the following:

We are given a table of **customers** and **products** they interacted with, however, some customers do not leave ratings on the product that they interacted with. In this case, how do we find these missing ratings?

Furthermore, let us say that we track certain features for the customers and products. For example:

Let $U$ denote the customers, and let $V$ denote items bought at a food mart. Let us say that $U$, $V$ contains **2 features**, sweet and savory, and let $A$ be the rating of sweet or savory items (from 1-5) by a customer that purchased them. Sweet and savory are what we refer to as **latent features**, and they relate customers to products as they are keys that merge customers and products. 

Now assume $A$ is the following customer-product interaction table...

$$A = \begin{pmatrix} 
4 & 0 & 3 & 2 & 2 & 1\\[3pt]
0 & 5 & 3 & 4 & 5 & 1\\[3pt]
5 & 5 & 0 & 0 & 3 & 5\\[3pt]
3 & 3 & 4 & 3 & 2 & 0\\[3pt]
2 & 4 & 2 & 4 & 5 & 1\\[3pt]
0 & 0 & 5 & 3 & 3 & 4
\end{pmatrix}$$

Then we want to find $U$ and $V$ such that:

- $U$ encodes the latent features (sweet or savory) of the customers, and $V$ does the same for the products.
- $U \cdot V^{T} \approx A$.

Thus, we are looking for $U$, $V$ such that:

$$A \approx U\cdot V^{T}$$

Now refer to the previous section for the mathematical details of our machine learning problem.


## Manual Implementation of Matrix Factorization

By "manual implementation", we mean *implementation in python with only numpy*. This is possible because we are using the following loss:

$$\mathrm{Loss} = \mathrm{MSE} + \mathrm{Regularization}$$

So we have a closed-form expression for the gradient of the loss in this case. If we wanted to implement more complicated loss functions, we will likely need to use an automatic differentiation package.

In [1]:
import numpy as np

In [2]:
#ARGUMENTS
# A - ground truth matrix
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization(A, U, V, k, steps, lmbda=0.0002, beta = 0.02):
    V = V.T

    for epoch in range(steps):
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                # we are only looking to minimize the loss on the already-filled entries
                if A[i][j] > 0:
                    # error
                    eij = A[i][j] - np.dot(U[i, :], V[:, j])
                    
                    for K in range(k):
                        # gradient descent step
                        U[i][K] = U[i][K] + lmbda * (2*eij * V[K][j] - beta*U[i][K])
                        V[K][j] = V[K][j] + lmbda * (2*eij * U[i][K] - beta*V[K][j])
                        
        er = np.dot(U,V)
        
        # e is our loss (i.e. our e_{ij}^{2})
        e = 0
        
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                if A[i][j] > 0:
                    # update the loss
                    e = e + (A[i][j] - np.dot(U[i,:], V[:,j]))**2
                    
                    for K in range(k):
                        # put regularization term on loss so it doesn't overfit on training data
                        e = e + (beta/2) * (U[i][K]**2 + V[K][j]**2)
                     
        # give print output of training (print only every 10 epochs)
        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

In [3]:
A = np.array([
    [5,3,2,1,0],
    [3,0,4,2,1],
    [4,0,0,1,0],
    [0,0,2,4,0],
    [5,3,0,0,1]
])

m = A.shape[0]
n = A.shape[1]
user_features = 2

np.random.seed(24)

# generate matrices with entries of random numbers 0 - 1
U = np.random.rand(m, user_features)
V = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U)
print('V is:\n', V)

# perform factorization

U_hat, V_hat = matrix_factorization(A, U, V, user_features, 200)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

U is:
 [[0.9600173  0.69951205]
 [0.99986729 0.2200673 ]
 [0.36105635 0.73984099]
 [0.99645573 0.31634698]
 [0.13654458 0.38398001]]
V is:
 [[0.32051928 0.36641475]
 [0.70965156 0.90014243]
 [0.53411544 0.24729376]
 [0.67180656 0.56172911]
 [0.54255988 0.8934476 ]]
Epoch 1 : Total Loss 99.275
Epoch 11 : Total Loss 96.767
Epoch 21 : Total Loss 94.202
Epoch 31 : Total Loss 91.584
Epoch 41 : Total Loss 88.918
Epoch 51 : Total Loss 86.207
Epoch 61 : Total Loss 83.459
Epoch 71 : Total Loss 80.680
Epoch 81 : Total Loss 77.878
Epoch 91 : Total Loss 75.059
Epoch 101 : Total Loss 72.233
Epoch 111 : Total Loss 69.408
Epoch 121 : Total Loss 66.595
Epoch 131 : Total Loss 63.801
Epoch 141 : Total Loss 61.036
Epoch 151 : Total Loss 58.311
Epoch 161 : Total Loss 55.633
Epoch 171 : Total Loss 53.012
Epoch 181 : Total Loss 50.455
Epoch 191 : Total Loss 47.970
U_hat is:
 [[1.2427563  0.99404657]
 [1.32958304 0.4449768 ]
 [0.53087359 0.91554011]
 [1.23935835 0.47920855]
 [0.51268872 0.8134178 ]]
V_hat is

This does not look good after 200 epochs! Let's try more epochs, and if that doesn't work, let's increase the learning rate.

In [4]:
U_hat, V_hat = matrix_factorization(A, U, V, user_features, 5000)

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

Epoch 1 : Total Loss 45.564
Epoch 11 : Total Loss 43.244
Epoch 21 : Total Loss 41.013
Epoch 31 : Total Loss 38.877
Epoch 41 : Total Loss 36.838
Epoch 51 : Total Loss 34.898
Epoch 61 : Total Loss 33.060
Epoch 71 : Total Loss 31.323
Epoch 81 : Total Loss 29.687
Epoch 91 : Total Loss 28.150
Epoch 101 : Total Loss 26.711
Epoch 111 : Total Loss 25.366
Epoch 121 : Total Loss 24.114
Epoch 131 : Total Loss 22.950
Epoch 141 : Total Loss 21.870
Epoch 151 : Total Loss 20.871
Epoch 161 : Total Loss 19.947
Epoch 171 : Total Loss 19.095
Epoch 181 : Total Loss 18.310
Epoch 191 : Total Loss 17.587
Epoch 201 : Total Loss 16.923
Epoch 211 : Total Loss 16.312
Epoch 221 : Total Loss 15.751
Epoch 231 : Total Loss 15.236
Epoch 241 : Total Loss 14.763
Epoch 251 : Total Loss 14.328
Epoch 261 : Total Loss 13.929
Epoch 271 : Total Loss 13.563
Epoch 281 : Total Loss 13.225
Epoch 291 : Total Loss 12.914
Epoch 301 : Total Loss 12.628
Epoch 311 : Total Loss 12.364
Epoch 321 : Total Loss 12.120
Epoch 331 : Total Los

Epoch 2801 : Total Loss 5.536
Epoch 2811 : Total Loss 5.535
Epoch 2821 : Total Loss 5.534
Epoch 2831 : Total Loss 5.533
Epoch 2841 : Total Loss 5.532
Epoch 2851 : Total Loss 5.531
Epoch 2861 : Total Loss 5.531
Epoch 2871 : Total Loss 5.530
Epoch 2881 : Total Loss 5.529
Epoch 2891 : Total Loss 5.528
Epoch 2901 : Total Loss 5.527
Epoch 2911 : Total Loss 5.526
Epoch 2921 : Total Loss 5.526
Epoch 2931 : Total Loss 5.525
Epoch 2941 : Total Loss 5.524
Epoch 2951 : Total Loss 5.523
Epoch 2961 : Total Loss 5.523
Epoch 2971 : Total Loss 5.522
Epoch 2981 : Total Loss 5.521
Epoch 2991 : Total Loss 5.521
Epoch 3001 : Total Loss 5.520
Epoch 3011 : Total Loss 5.519
Epoch 3021 : Total Loss 5.519
Epoch 3031 : Total Loss 5.518
Epoch 3041 : Total Loss 5.517
Epoch 3051 : Total Loss 5.517
Epoch 3061 : Total Loss 5.516
Epoch 3071 : Total Loss 5.516
Epoch 3081 : Total Loss 5.515
Epoch 3091 : Total Loss 5.515
Epoch 3101 : Total Loss 5.514
Epoch 3111 : Total Loss 5.513
Epoch 3121 : Total Loss 5.513
Epoch 3131

After training for long enough, we see that the existing interactions are predicted quite well. To see the average error of each entry in $A_{hat}$ corresponding to NONZEROS in $A$, we can just use the following:

In [5]:
A_hat = np.dot(U_hat, V_hat.T)
A_hat

array([[4.98447014, 2.97009182, 1.54598166, 1.50219606, 0.9902487 ],
       [2.99605858, 2.12761136, 3.17161116, 2.87072417, 0.95654322],
       [3.98158649, 2.34293891, 1.04127825, 1.03001406, 0.75980498],
       [4.52329661, 2.9474781 , 3.05471784, 2.81272953, 1.16479036],
       [4.96573884, 3.01061644, 1.87870719, 1.79363369, 1.0410779 ]])

In [6]:
A

array([[5, 3, 2, 1, 0],
       [3, 0, 4, 2, 1],
       [4, 0, 0, 1, 0],
       [0, 0, 2, 4, 0],
       [5, 3, 0, 0, 1]])

In [7]:
# find nonzero indices
nonzero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i][j] > 0]

nonzero_ind

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 3),
 (3, 2),
 (3, 3),
 (4, 0),
 (4, 1),
 (4, 4)]

In [8]:
# compute average error for all entries in A_hat corresponding to nonzeros in A
avg_error = sum([(A_hat[i][j] - A[i][j])**2 for i, j in nonzero_ind])/len(nonzero_ind)

print(f'The mean squared error of all predictions is {avg_error:.3f}.')
print(f'The root mean squared error of all predictions is {np.sqrt(avg_error):.3f}.')

The mean squared error of all predictions is 0.295.
The root mean squared error of all predictions is 0.544.


Therefore, our matrix factorization algorithm is very good at predicting the existing entries. Let us see what the predictions were for the empty entries.

In [9]:
# find zero indices of A
zero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i][j] == 0]

zero_ind

[(0, 4),
 (1, 1),
 (2, 1),
 (2, 2),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 4),
 (4, 2),
 (4, 3)]

In [10]:
predictions = [[A_hat[i][j], f'At index:{(i,j)}'] for i,j in zero_ind]

predictions

[[0.990248697667072, 'At index:(0, 4)'],
 [2.1276113626791973, 'At index:(1, 1)'],
 [2.3429389126641804, 'At index:(2, 1)'],
 [1.0412782499755882, 'At index:(2, 2)'],
 [0.7598049813295886, 'At index:(2, 4)'],
 [4.523296605425644, 'At index:(3, 0)'],
 [2.947478096798725, 'At index:(3, 1)'],
 [1.1647903631780367, 'At index:(3, 4)'],
 [1.8787071873760717, 'At index:(4, 2)'],
 [1.793633687301854, 'At index:(4, 3)']]

To take it one step further, we could even round these up or down in order to give a good prediction for what these interactions *would have* been rated.

In [11]:
rounded_predictions = [[int(np.round(A_hat[i][j], decimals=0)), f'At index:{(i,j)}'] for i,j in zero_ind]

rounded_predictions

[[1, 'At index:(0, 4)'],
 [2, 'At index:(1, 1)'],
 [2, 'At index:(2, 1)'],
 [1, 'At index:(2, 2)'],
 [1, 'At index:(2, 4)'],
 [5, 'At index:(3, 0)'],
 [3, 'At index:(3, 1)'],
 [1, 'At index:(3, 4)'],
 [2, 'At index:(4, 2)'],
 [2, 'At index:(4, 3)']]

**Non-Square Matrix Test Case**

In [12]:
A = np.array([
    [5,3,0,1],
    [3,2,0,0],
    [0,1,4,2],
    [1,0,0,5]
])

m, n = A.shape[0], A.shape[1]

user_features = 2

np.random.seed(42)

U_1 = np.random.rand(m, user_features)
V_1 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_1)
print('V is:\n', V_1)

# perform factorization

U_hat, V_hat = matrix_factorization(A, U_1, V_1, user_features, 5000)

A_hat = np.dot(U_hat, V_hat.T)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A
print('A is:\n', A)
print('A_hat is:\n', A_hat)


U is:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]]
V is:
 [[0.60111501 0.70807258]
 [0.02058449 0.96990985]
 [0.83244264 0.21233911]
 [0.18182497 0.18340451]]
Epoch 1 : Total Loss 70.905
Epoch 11 : Total Loss 69.765
Epoch 21 : Total Loss 68.610
Epoch 31 : Total Loss 67.441
Epoch 41 : Total Loss 66.260
Epoch 51 : Total Loss 65.068
Epoch 61 : Total Loss 63.867
Epoch 71 : Total Loss 62.659
Epoch 81 : Total Loss 61.447
Epoch 91 : Total Loss 60.231
Epoch 101 : Total Loss 59.014
Epoch 111 : Total Loss 57.799
Epoch 121 : Total Loss 56.587
Epoch 131 : Total Loss 55.380
Epoch 141 : Total Loss 54.182
Epoch 151 : Total Loss 52.994
Epoch 161 : Total Loss 51.819
Epoch 171 : Total Loss 50.657
Epoch 181 : Total Loss 49.513
Epoch 191 : Total Loss 48.386
Epoch 201 : Total Loss 47.280
Epoch 211 : Total Loss 46.195
Epoch 221 : Total Loss 45.134
Epoch 231 : Total Loss 44.098
Epoch 241 : Total Loss 43.087
Epoch 251 : Total Loss 42.103
Epoch 261 : Tot

Epoch 2851 : Total Loss 2.990
Epoch 2861 : Total Loss 2.986
Epoch 2871 : Total Loss 2.982
Epoch 2881 : Total Loss 2.979
Epoch 2891 : Total Loss 2.975
Epoch 2901 : Total Loss 2.972
Epoch 2911 : Total Loss 2.968
Epoch 2921 : Total Loss 2.965
Epoch 2931 : Total Loss 2.961
Epoch 2941 : Total Loss 2.958
Epoch 2951 : Total Loss 2.955
Epoch 2961 : Total Loss 2.952
Epoch 2971 : Total Loss 2.949
Epoch 2981 : Total Loss 2.946
Epoch 2991 : Total Loss 2.943
Epoch 3001 : Total Loss 2.940
Epoch 3011 : Total Loss 2.938
Epoch 3021 : Total Loss 2.935
Epoch 3031 : Total Loss 2.932
Epoch 3041 : Total Loss 2.930
Epoch 3051 : Total Loss 2.927
Epoch 3061 : Total Loss 2.925
Epoch 3071 : Total Loss 2.922
Epoch 3081 : Total Loss 2.920
Epoch 3091 : Total Loss 2.917
Epoch 3101 : Total Loss 2.915
Epoch 3111 : Total Loss 2.912
Epoch 3121 : Total Loss 2.910
Epoch 3131 : Total Loss 2.908
Epoch 3141 : Total Loss 2.906
Epoch 3151 : Total Loss 2.903
Epoch 3161 : Total Loss 2.901
Epoch 3171 : Total Loss 2.899
Epoch 3181

## TensorFlow Implementation of Matrix Factorization (Pending)

We can take advantage of TensorFlow's automatic differentation to perform this for us. 

One major step that was a weakness for us in the previous implementation was that **we needed to solve for the gradient of each entry of the error matrix by hand**. While this is easily the most computationally cost-effective way to do this, we would like to compute a number for the gradient as we go instead of obtaining the closed-form expression and then plugging our values in.

(**Not Done Yet**)

In [13]:
# import tensorflow as tf

We will now be implementing this from scratch on tensorflow. One reason why optimizing to run this in tensorflow would be more effective is that we can take advantage of **graph execution**. If we can somehow get the error computation in a graph, it would be extremely quick.

In [14]:
# A = np.array([
#     [5,3,2,1,0],
#     [3,0,4,2,1],
#     [4,0,0,1,0],
#     [0,0,2,4,0],
#     [5,3,0,0,1]
# ], dtype=np.float32)

# m = A.shape[0]
# n = A.shape[1]
# user_features = 2

# np.random.seed(24)

# # generate matrices with entries of random numbers 0 - 1
# U = np.random.rand(m, user_features)
# V = np.random.rand(n, user_features)

# U, V

(array([[0.9600173 , 0.69951205],
        [0.99986729, 0.2200673 ],
        [0.36105635, 0.73984099],
        [0.99645573, 0.31634698],
        [0.13654458, 0.38398001]]),
 array([[0.32051928, 0.36641475],
        [0.70965156, 0.90014243],
        [0.53411544, 0.24729376],
        [0.67180656, 0.56172911],
        [0.54255988, 0.8934476 ]]))

In [15]:
# A = tf.convert_to_tensor(A, dtype = tf.float32)

# U = tf.convert_to_tensor(U, dtype=tf.float32)

# V = tf.convert_to_tensor(V, dtype=tf.float32)

In [16]:
# A, U, V

(<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[5., 3., 2., 1., 0.],
        [3., 0., 4., 2., 1.],
        [4., 0., 0., 1., 0.],
        [0., 0., 2., 4., 0.],
        [5., 3., 0., 0., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[0.9600173 , 0.69951206],
        [0.9998673 , 0.22006729],
        [0.36105636, 0.739841  ],
        [0.9964557 , 0.31634697],
        [0.13654459, 0.38398   ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[0.3205193 , 0.36641476],
        [0.7096516 , 0.90014243],
        [0.53411543, 0.24729377],
        [0.6718066 , 0.56172913],
        [0.54255986, 0.8934476 ]], dtype=float32)>)

In [17]:
# U[1,:]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.9998673 , 0.22006729], dtype=float32)>

In [18]:
# V_t = tf.transpose(V)

# tf.einsum('i,i->',U[1,:],V_t[:,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.90764934>

In [ ]:
# V = tf.transpose(V)

# m = tf.einsum('ij,jk->ik',U, V)

# m

In [25]:
## TensorFlow Implementation Pending (likely not possible directly)

# A = np.array([
#     [5,3,2,1,0],
#     [3,0,4,2,1],
#     [4,0,0,1,0],
#     [0,0,2,4,0],
#     [5,3,0,0,1]
# ], dtype=np.float32)

# steps = 200
# m = A.shape[0]
# n = A.shape[1]
# user_features = 2

# np.random.seed(24)

# # generate matrices with entries of random numbers 0 - 1
# U = np.random.rand(m, user_features)
# V = np.random.rand(n, user_features)

# # convert A, U, V into tf tensors for input
# A = tf.convert_to_tensor(A, dtype = tf.float32)

# U = tf.convert_to_tensor(U, dtype=tf.float32)

# V = tf.convert_to_tensor(V, dtype=tf.float32)

# # transpose V
# V = tf.transpose(V)
# # set optimizer for gradient descent
# optimizer = tf.keras.optimizers.SGD(0.01)
# # number of latent features
# K = 2

# @tf.function
# def component_wise_mse(A, comp1, comp2):
#     eij = A[i,j] - comp1*comp2
#     return tf.pow(eij, 2)


# for epoch in range(steps):
#     for i in range(A.shape[0]):
#         for j in range(A.shape[1]):
#             # we are only looking to minimize the loss on the already-filled entries
#             if A[i][j] > 0:
#                 for k in range(K):
#                     # error                
#                     v_kj = tf.Variable(V[k, j], trainable=True)
#                     u_ik = tf.Variable(U[i, k], trainable=True)
#                     trainable = [u_ik, v_kj]
#                     with tf.GradientTape() as tp:
#                         # define error (loss) in terms of trainable variables
#                         loss_fn = component_wise_mse(A, u_ik, v_kj)

#                         dloss = tp.gradient(loss_fn, trainable)
#     #                     # gradient descent step
#     #                     optimizer.apply_gradients(zip(dloss, loss_fn))
#     print(epoch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
